In [1]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [3]:
iris = load_iris()
X, y = iris.data, iris.target
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [5]:
X_train, X_test, y_train , y_test = train_test_split(X, y, random_state = 42, test_size = 0.3)  

In [7]:
# Antibody classes
class Antibody:
    def __init__(self, n_features = 4, n_classes = 3):
        self.weights = np.random.randn(n_classes, n_features)
        self.bias = np.random.randn(n_classes)
        self.affinity = 0.0

    def predict(self, x):
        logits = np.dot(self.weights, x) + self.bias
        return np.argmax(logits)

    def evaluate(self, X, y, reg_lambda=0.01):
        preds = [self.predict(x) for x in X]
        acc = accuracy_score(y, preds)
        reg_term = reg_lambda * np.sum(self.weights**2)
        self.affinity = acc - reg_term
        return self.affinity

In [9]:
# CLONALG with regularization + early stopping
def clonal_selection(
    X, y, pop_size = 20, n_generations = 50, clones_per = 5, mutation_rate = 0.1, reg_lambda = 0.01, patience=5
):
    population = [Antibody() for _ in range(pop_size)]
    patience_counter = 0
    best_affinity = -np.inf

    for gen in range(n_generations):
        for ab in population:
            ab.evaluate(X,y,reg_lambda)

        population.sort(key=lambda ab:ab.affinity, reverse=True)
        best = population[:pop_size // 2]

        if best[0].affinity > best_affinity + 1e-4:
            best_affinity = best[0].affinity
            patience_counter = 0
        else:
            patience_counter += 1

        print(f'Generation {gen+1} - Best_affinity: {best_affinity:.4f}')
        if patience_counter >= patience:
            print("Early stopping due to environment")
            break
            
        # Cloning and mutation
        clones=[]
        for ab in best:
            for _ in range(clones_per):
                clone = Antibody()
                clone.weights = ab.weights + np.random.normal(0, mutation_rate, size=ab.weights.shape)
                clone.bias = ab.bias + np.random.normal(0, mutation_rate, size=ab.bias.shape)
                clones.append(clone)

        for clone in clones:
            clone.evaluate(X, y, reg_lambda)

        population = sorted(best + clones, key=lambda ab: ab.affinity, reverse=True)
    return population[0]

In [11]:
# Training
best_model = clonal_selection(X_train, y_train)

# Evaluation
preds = [best_model.predict(x) for x in X_test]
acc = accuracy_score(y_test, preds)
print(f"Final Test accuracy: {acc * 100}")

Generation 1 - Best_affinity: 0.6224
Generation 2 - Best_affinity: 0.6365
Generation 3 - Best_affinity: 0.6845
Generation 4 - Best_affinity: 0.7345
Generation 5 - Best_affinity: 0.7440
Generation 6 - Best_affinity: 0.7747
Generation 7 - Best_affinity: 0.7948
Generation 8 - Best_affinity: 0.8123
Generation 9 - Best_affinity: 0.8489
Generation 10 - Best_affinity: 0.8489
Generation 11 - Best_affinity: 0.8540
Generation 12 - Best_affinity: 0.8762
Generation 13 - Best_affinity: 0.8762
Generation 14 - Best_affinity: 0.8970
Generation 15 - Best_affinity: 0.9196
Generation 16 - Best_affinity: 0.9196
Generation 17 - Best_affinity: 0.9196
Generation 18 - Best_affinity: 0.9354
Generation 19 - Best_affinity: 0.9354
Generation 20 - Best_affinity: 0.9354
Generation 21 - Best_affinity: 0.9354
Generation 22 - Best_affinity: 0.9354
Generation 23 - Best_affinity: 0.9354
Early stopping due to environment
Final Test accuracy: 97.77777777777777
